In [3]:
import numpy as np
from scipy.ndimage import label, generate_binary_structure
from scipy import ndimage

In [7]:
def measure_box_dimensions(box_mask, floor_plane, top_plane):
    # calculate hight : distance between the 2 plains
    height = abs(top_plane[3] - floor_plane[3]) / np.linalg.norm(floor_plane[:3])
    # calculate lenght and width: 1. find corners
                                #2. find coordinates of the corners
                                #3. subtract the coordinates.


    #about 1.) find corners we need a intersection line from both planes
    intersection_line = np.cross(floor_plane[:3], top_plane[:3])

    # Normalize the intersection line
    intersection_line /= np.linalg.norm(intersection_line)
    #finding box corners
    #1)findining first for box corner from pixel
    #2)than intersecting the line of  from box plane to surace plane
    # Define parameters for the Harris corner detection
    k = 0.04  # Harris corner constant (adjust as needed)
    window_size = 3  # Size of the window for local analysis
    threshold = 5000  # Adjust this threshold as needed

    ############################
#  TOP box corners via pixel
    # Calculate image gradients
    dx = PC[:]# x elements
    dy = PC[:]# y elements

    Ix = convolve(PC,dx)
    Iy = convolve(PC, dy)

    # Calculate components of the Harris matrix M
    Ix2 = Ix * Ix
    Iy2 = Iy * Iy
    IxIy = Ix * Iy

    # Calculate the sums of squared differences within the window
    A = convolve(Ix2, np.ones((window_size, window_size)))
    B = convolve(Iy2, np.ones((window_size, window_size)))
    C = convolve(IxIy, np.ones((window_size, window_size)))

    # Calculate the determinant and trace of M for each pixel
    T = np.array([[A,C],[C,B]])
    trace_M = np.trace(T)
    det_M = np.linalg.det(T.T)
    trace_M = trace_M.T

    # Calculate the Harris response for each pixel
    harris_response = det_M - k * (trace_M**2)

    # Threshold the Harris response to find corners
    corners = np.argwhere(harris_response > threshold)

    ############################################
##2) finding the corner at bootom part of corner of box with intersecting lines on surace plane

    bottom_box_corners = []
    for i in range(4):
        # For each corner of the box, compute the intersection point with the intersection line
        corner = np.array([x[i], y[i], z[i]])  # Replace with the coordinates of the box's corners
        t = -(np.dot(intersection_line, corner) + intersection_line[3]) / np.dot(intersection_line, intersection_line)
        intersection_point = corner + t * intersection_line
        bottom_box_corners.append(intersection_point)

    return bottom_box_corners

    # Calculate the length, width, and depth of the box
    length = np.linalg.norm(box_corners[0] - box_corners[1])
    width = np.linalg.norm(box_corners[1] - box_corners[2])
    depth = np.linalg.norm(box_corners[0] - box_corners[3])

    return {"height": height,"length": length,"width": width,"depth": depth}
    pass

In [ ]:
# scipy.ndimage.label(box_top_detection)
labeled_array, num_features = ndimage.label(box_top_detection)

# Find the largest connected component
unique_labels, label_counts = np.unique(labeled_array, return_counts=True)

# Exclude label 0, which represents the background
label_counts = label_counts[1:]
largest_component_label = unique_labels[np.argmax(label_counts)]

# Create a mask for the largest component
largest_component_mask = labeled_array == largest_component_label